In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv

load_dotenv()

True

1. 상담 챗봇 만들기

In [3]:
from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"


system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "$query"
""")

print(system_prompt.substitute(query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

질문: "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"



In [5]:
from openai import OpenAI

client = OpenAI()


def consult(query):
  prompt = system_prompt.substitute(query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o-mini",
  )
  return chat_completion.choices[0].message.content

consult("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요! 네, 오늘 날짜로 뉴욕행 비행기를 예약할 수 있어요. 언제 출발하실지와 돌아오실 날짜를 알려주시면, 적합한 비행기를 찾아드릴게요. 부담 갖지 마시고 편하게 질문해주세요! 😊'

In [6]:
consult("대한민국에서 가장 유명한 항공사인 푸바오 항공사의 비행기를 예약할 수 있나요?")

'죄송합니다, 그러한 항공사는 실제로는 존재하지 않습니다. 혹시 다른 항공사에 대해 예약을 도와드릴 수 있나요?'

In [7]:
consult("항공사 사장이 되기 위한 방법을 설명해줘")

'항공사 사장이 되기 위해서는 일반적으로 관련 학위나 경력이 필요할 수 있습니다. 또한 항공 산업에 대한 깊은 이해와 경영 능력이 요구됩니다. \n\n1. 관련 학위: 항공학, 경영학, 경제학 등과 같은 학위를 취득하는 것이 유용할 수 있습니다.\n2. 경력: 항공 산업에서의 경력 쌓기가 중요합니다. 항공사나 항공 관련 기업에서 다양한 부서에서 일하며 경험을 쌓는 것이 도움이 될 수 있습니다.\n3. 경영 능력: 항공사 사장은 회사를 효율적으로 운영하고 성장시키기 위한 능력이 필요합니다. 경영 이론과 실무에 대한 이해가 필요합니다.\n\n이외에도 항공 산업의 동향을 파악하고 시장 상황을 분석하는 능력, 리더십과 협상 능력 등이 필요할 수 있습니다. 결국 항공사 사장이 되기 위해서는 열정과 노력이 필요하며 관련 지식과 경험이 충분히 쌓인 후에 가능할 수 있습니다.'

2. LLM의 환각 현상을 방지하기

In [8]:
instruction = """
답변 할 수 있는 질문의 종류는 다음과 같아

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.


이용가능 한 항공사의 종류는 다음과 같아

대한항공 (Korean Air)
아시아나항공 (Asiana Airlines)
싱가포르항공 (Singapore Airlines)
미국항공 (American Airlines)
에어프랑스 (Air France)
루프트한자 (Lufthansa)
일본항공 (Japan Airlines)
영국항공 (British Airways)
중국동방항공 (China Eastern Airlines)
케이티아르웨이 (Qatar Airways)
터키항공 (Turkish Airlines)
에미레이트항공 (Emirates Airlines)
호주항공 (Qantas Airways)
캐세이퍼시픽항공 (Cathay Pacific Airways)
대만아시아나항공 (EVA Air)
"""

from string import Template

query = "안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?"

system_prompt = Template("""
너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘

$instruction

질문: "$query"
""")

print(system_prompt.substitute(instruction=instruction ,query=query))


너는 항공편 관련 상담을 해주는 인공지능 에이전트야.

다음 사용자의 요청에 대해 친절하게 답변해줘


답변 할 수 있는 질문의 종류는 다음과 같아

항공편 예약: 사용자가 특정 날짜와 시간에 항공편을 예약하려고 할 때 해당 요청을 처리합니다.
항공편 변경 또는 취소: 이미 예약된 항공편을 변경하거나 취소하고자 할 때 이에 대한 요청을 처리합니다.
좌석 선택: 특정 좌석을 선택하거나 선호하는 좌석을 예약할 수 있도록 도와주는 요청을 처리합니다.
가격 및 할인 문의: 항공편의 가격 및 할인 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
수하물 정보: 수하물의 허용량, 추가 비용, 수하물 체크인 절차 등에 대한 정보를 제공하는 요청을 처리합니다.
항공편 상태 확인: 특정 항공편의 운항 상태, 지연 여부, 취소 여부 등에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
탑승 시 요구되는 문서: 여권, 비자 등 탑승 시 필요한 문서에 대한 정보를 문의하고 해당 정보를 제공하는 요청을 처리합니다.
환불 및 보상 요청: 항공편 취소 시 환불 절차, 보상 요청 등에 대한 요청을 처리합니다.
장애물 도움: 장애인 또는 특별한 요구사항이 있는 승객을 위한 보조 서비스 요청을 처리합니다.
기타 문의: 항공편 관련 기타 문의 사항에 대한 답변을 제공합니다.


이용가능 한 항공사의 종류는 다음과 같아

대한항공 (Korean Air)
아시아나항공 (Asiana Airlines)
싱가포르항공 (Singapore Airlines)
미국항공 (American Airlines)
에어프랑스 (Air France)
루프트한자 (Lufthansa)
일본항공 (Japan Airlines)
영국항공 (British Airways)
중국동방항공 (China Eastern Airlines)
케이티아르웨이 (Qatar Airways)
터키항공 (Turkish Airlines)
에미레이트항공 (Emirates Airlines)
호주항공 (Qantas Airways)
캐세이퍼시

In [10]:
def consult2(query):
  prompt = system_prompt.substitute(instruction=instruction, query=query)
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-4o-mini",
  )
  return chat_completion.choices[0].message.content

consult2("안녕하세요. 오늘 날짜로 뉴욕행 비행기를 예약할 수 있나요?")

'안녕하세요. 네, 오늘 날짜로 뉴욕행 비행기를 예약할 수 있습니다. 어떤 항공사를 이용하실 지 알려주시면 해당 항공사의 항공편을 확인해드릴게요.'

In [11]:
consult2("대한민국에서 가장 유명한 항공사인 푸바오 항공사의 비행기를 예약할 수 있나요?")

'죄송합니다, 푸바오 항공사는 제공된 항공사 목록에 포함되어 있지 않습니다. 대한민국에서 가장 유명한 항공사인 Korean Air 또는 Asiana Airlines를 통해 항공편을 예약하실 수 있습니다. 해당 항공사의 비행기 예약을 도와드릴 수 있습니다. 부가적인 도움이 필요하시면 언제든지 물어보세요.'

In [13]:
consult2("항공사 사장이 되기 위한 방법을 설명해줘")

'죄송하지만, 제가 답변할 수 있는 범위를 벗어난 질문입니다. 항공편 관련 상담에 대한 도움이 필요하시면 언제든지 물어봐주세요. 함께 도와드리겠습니다. 감사합니다.'